각 test_ID에 교통사고 값(ECLO)을 예측하기 

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1

- 출퇴근 시간, 새벽 시간(음주, 졸음)에 많이 발생?

- 요일에 따라 달라진다? -> 인구 밀집

- 날씨 영향?

- 사고가 빈번한 지역?

- cctv가 없거나 신호 시간이 짧거나

- 도로 상태, 형태

- 노인이 많으면 사고가 자주 발생

# 카테코리로 묶어 xgb, lgbm 실행하기
# 조기종료

# 라이브러리 불러오기

In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
accident_df = pd.read_csv('./data/external_open/countrywide_accident.csv')

train_df.head(3)
train_df.shape

test_df.head(3)
test_df.shape

accident_df.head(3)
accident_df.shape

(602775, 23)

In [3]:
# 추가할 수 있는 data

# cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv', encoding = 'cp949')
# security_light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv', encoding = 'cp949', low_memory = False)
# child_protection_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv', encoding = 'cp949')
# parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv', encoding = 'cp949')

# cctv_df.head(2)
# security_light_df.head(2)
# child_protection_df.head(2)
# parking_df.head(2)

# EDA

## 결측치 확인

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 약 1000개의 결측치 존재

In [5]:
test_df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB


In [6]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602775 entries, 0 to 602774
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           602775 non-null  object
 1   사고일시         602775 non-null  object
 2   요일           602775 non-null  object
 3   기상상태         602775 non-null  object
 4   시군구          602775 non-null  object
 5   도로형태         602775 non-null  object
 6   노면상태         602774 non-null  object
 7   사고유형         602775 non-null  object
 8   사고유형 - 세부분류  602775 non-null  object
 9   법규위반         602775 non-null  object
 10  가해운전자 차종     602775 non-null  object
 11  가해운전자 성별     602775 non-null  object
 12  가해운전자 연령     602775 non-null  object
 13  가해운전자 상해정도   602775 non-null  object
 14  피해운전자 차종     576948 non-null  object
 15  피해운전자 성별     576946 non-null  object
 16  피해운전자 연령     576949 non-null  object
 17  피해운전자 상해정도   576946 non-null  object
 18  사망자수         602775 non-null  int64 
 19  중상

=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 결측치가 존재하는 이유 -> '사고 유형'이 '챠랑단독'이기 때문이다

=> 이런 결측치는 어떻게 처리를 해야하나

## 변수별 유의미한지 확인

In [7]:
# 요일별로 언제 사고가 많이 발생하는지

train_df['요일'].value_counts() # 금요일이 제일 많이 발생

요일
금요일    6179
화요일    6023
수요일    5969
월요일    5895
목요일    5759
토요일    5670
일요일    4114
Name: count, dtype: int64

In [8]:
# 기상상태로 확인

train_df.groupby(['기상상태'])[['사망자수', '중상자수', '경상자수', '부상자수']].sum() # 맑을 때? 눈 올때가 많은 줄 알았는데

,사망자수,중상자수,경상자수,부상자수
기상상태,,,,
기타,1,22,50,7
눈,1,1,7,1
맑음,265,9383,38781,4609
비,36,763,2788,322
안개,0,1,13,0
흐림,5,222,746,86


In [9]:
# 도로형태

train_df['도로형태'].value_counts()

도로형태
단일로 - 기타           19485
교차로 - 교차로안          9931
교차로 - 교차로부근         5746
기타 - 기타             1986
교차로 - 교차로횡단보도내      1474
단일로 - 지하차도(도로)내      309
주차장 - 주차장            236
단일로 - 교량             232
단일로 - 고가도로위          138
단일로 - 터널              64
미분류 - 미분류              8
Name: count, dtype: int64

In [10]:
# 노면상태

train_df['노면상태'].value_counts()

노면상태
건조       36420
젖음/습기     3100
기타          58
서리/결빙       25
침수           3
적설           3
Name: count, dtype: int64

In [11]:
# 사고유형

train_df['사고유형'].value_counts()

사고유형
차대차     31785
차대사람     6833
차량단독      991
Name: count, dtype: int64

# 데이터 전처리

## train에 있고 test에 없는 값 삭제

In [12]:
train_df['기상상태'].unique()
accident_df['기상상태'].unique()
test_df['기상상태'].unique()

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

In [13]:
train_df['노면상태'].unique()
accident_df['노면상태'].unique()
test_df['노면상태'].unique()

array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

In [14]:
train_df['사고유형'].unique()
accident_df['사고유형'].unique()
test_df['사고유형'].unique()

array(['차대사람', '차대차', '차량단독'], dtype=object)

=> train, accident '기상상태'에는 안개가 있고 test '기상상태'에는 안개가 없어 삭제

=> accident '노면상태'에는 해빙, nan가 있고 train test에는 없어 삭제

=> accident '사고유형'에는 철길건널목이 있고 train, test에는 없어 삭제

In [15]:
# '기상상태'에서 '안개' 삭제

train_df = train_df[train_df['기상상태'] != '안개']
accident_df = accident_df[accident_df['기상상태'] != '안개']

train_df['기상상태'].unique()
accident_df['기상상태'].unique()

array(['맑음', '흐림', '눈', '기타', '비'], dtype=object)

In [16]:
# '노면상태'에서 '해빙', nan 삭제

accident_df = accident_df[accident_df['노면상태'] != '해빙']
accident_df = accident_df[~accident_df['노면상태'].isnull()]

accident_df['노면상태'].unique()

array(['건조', '젖음/습기', '기타', '서리/결빙', '적설', '침수'], dtype=object)

In [17]:
# '사고유형'에서 '철길건널목' 삭제

accident_df = accident_df[accident_df['사고유형'] != '철길건널목']

accident_df['사고유형'].unique()

array(['차대사람', '차대차', '차량단독'], dtype=object)

## 시군구, 날짜, 도로형태, 사고 유형 분리 분리하기

In [18]:
# accident_df에 '시군구'를 나눌 때 결측치가 발생하므로 따로 처리

for i in range(len(accident_df)):
    name = accident_df.iloc[i]['시군구']
    name_lst = name.split()

    if len(name_lst) == 2:
        re_name = f'{name_lst[0]}{name_lst[0]}{name_lst[0]}'
        accident_df.loc[i, '시군구'] = re_name

In [19]:
# 시군구 나누는 함수 구현

def si_gun_gu(df):
    lst = df['시군구'].str.split(' ')
    df['시'] = lst.str[0]
    df['군'] = lst.str[1]
    df['구'] = lst.str[2]
    return df

In [20]:
train_df = si_gun_gu(train_df)
test_df = si_gun_gu(test_df)
accident_df = si_gun_gu(accident_df)

train_df.head(3)
test_df.head(3)
accident_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,시,군,구
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01 00,화요일,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,40세,경상,0.0,0.0,1.0,0.0,3.0,서울특별시,강서구,방화동
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01 00,화요일,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,20세,경상,0.0,0.0,1.0,0.0,3.0,경기도,포천시,소흘읍
2,COUNTRYWIDE_ACCIDENT_000002,2019-01-01 00,화요일,맑음,경기도 양주시 고암동,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,17세,중상,0.0,1.0,0.0,0.0,5.0,경기도,양주시,고암동


In [21]:
# 사고일시를 나누는 함수 

def year_mon_day(df):
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    df['연'] = df['사고일시'].dt.year
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['요일'] = df['사고일시'].dt.day_of_week # 0:월 ~ 6일요일
    df['시간'] = df['사고일시'].dt.hour
    return df

In [22]:
train_df = year_mon_day(train_df)
test_df = year_mon_day(test_df)
accident_df = year_mon_day(accident_df)

train_df.head(3)
test_df.head(3)
accident_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,경상자수,부상자수,ECLO,시,군,구,연,월,일,시간
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01,1.0,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,1.0,0.0,3.0,서울특별시,강서구,방화동,2019.0,1.0,1.0,0.0
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01,1.0,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,1.0,0.0,3.0,경기도,포천시,소흘읍,2019.0,1.0,1.0,0.0
2,COUNTRYWIDE_ACCIDENT_000002,2019-01-01,1.0,맑음,경기도 양주시 고암동,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,0.0,0.0,5.0,경기도,양주시,고암동,2019.0,1.0,1.0,0.0


In [23]:
# 도로형태를 '-'를 기준으로 분리하는 함수 구현
def road(df):
    lst = df['도로형태'].str.split(' - ')
    df['도로형태 - 대분류'] = lst.str[0]
    df['도로형태 - 소분류'] = lst.str[1]
    return df

In [24]:
train_df = road(train_df)
test_df = road(test_df)
accident_df = road(accident_df)

train_df.head(2)
test_df.head(2)
accident_df.head(2)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,ECLO,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류
0,COUNTRYWIDE_ACCIDENT_000000,2019-01-01,1.0,맑음,서울특별시 강서구 방화동,교차로 - 교차로횡단보도내,건조,차대사람,횡단중,보행자보호의무위반,...,3.0,서울특별시,강서구,방화동,2019.0,1.0,1.0,0.0,교차로,교차로횡단보도내
1,COUNTRYWIDE_ACCIDENT_000001,2019-01-01,1.0,맑음,경기도 포천시 소흘읍,교차로 - 교차로안,건조,차대차,기타,직진우회전진행방해,...,3.0,경기도,포천시,소흘읍,2019.0,1.0,1.0,0.0,교차로,교차로안


In [25]:
# 사고 유형 분리하기
# 사고 유형은 차대차, 차대 사람, 차량단독
def car_people(df):
    pattern = r'(\S)(\S)(\S+)'
    df[['사고유형1', '대', '사고유형2']] = df['사고유형'].str.extract(pattern)

    return df

In [26]:
train_df = car_people(train_df)
test_df = car_people(test_df)
accident_df = car_people(accident_df)

In [27]:
# 필요없는 col 제거
train_df = train_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)
test_df = test_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)
accident_df = accident_df.drop(['사고일시', '시군구', '도로형태', '사고유형', '대'], axis = 1)

In [28]:
train_df.head(1)
test_df.head(1)

,ID,요일,기상상태,노면상태,시,군,구,연,월,일,시간,도로형태 - 대분류,도로형태 - 소분류,사고유형1,사고유형2
0,ACCIDENT_39609,5,맑음,건조,대구광역시,수성구,상동,2022,1,1,1,교차로,교차로안,차,사람


In [29]:
train_df['사고유형1'].unique()
train_df['사고유형2'].unique()

array(['사람', '차', '단독'], dtype=object)

## 필요한 col만

In [30]:
X_test = test_df.drop(['ID'], axis = 1).copy()
X_train = train_df[X_test.columns].copy()
y_train = train_df['ECLO'].copy()

In [31]:
X_test.head(1)
X_train.head(1)
y_train.head(1)

0    5
Name: ECLO, dtype: int64

## 범주형 -> 수치형

In [32]:
# onehotencoding 이용
# X_train. X_test만 실행

num_col = ['요일', '연', '월', '일', '시간']
cat_col = ['기상상태', '노면상태', '시', '군', '구', '도로형태 - 대분류', '도로형태 - 소분류', '사고유형1', '사고유형2']
X = pd.concat([X_train, X_test])
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[cat_col])

X_train_res = ohe.transform(X_train[cat_col])
X_test_res = ohe.transform(X_test[cat_col])

X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names_out())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names_out())

X_train_res = pd.concat([X_train_ohe, X_train[num_col].reset_index(drop = True)], axis = 1)
X_test_res = pd.concat([X_test_ohe, X_test[num_col]], axis = 1)

In [33]:
X_train_res

,기상상태_기타,기상상태_눈,기상상태_맑음,기상상태_비,기상상태_흐림,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,...,도로형태 - 소분류_터널,사고유형1_차,사고유형2_단독,사고유형2_사람,사고유형2_차,요일,연,월,일,시간
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1,2019,1,1,0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1,2019,1,1,0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1,2019,1,1,1
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1,2019,1,1,2
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1,2019,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39596,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,4,2021,12,31,19
39597,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,4,2021,12,31,19
39598,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,4,2021,12,31,21
39599,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,4,2021,12,31,22


## 데이터 나누기

In [41]:
# X_tr, X_val, y_tr, y_val = train_test_split(X_train_res, y_train, test_size = 0.3)

# 모델링

In [44]:
# 조기종료
# evals = [(X_tr, y_tr), (X_val, y_val)]
xgb = XGBRegressor(
    learning_rate=0.1,          # 학습률
    random_state=42,
    verbosity=0,                # 로그 출력을 최소화합니다.
    n_estimators=100,           # 트리의 개수
    max_depth=6,                # 트리의 최대 깊이
    n_jobs=-1
#     early_stopping_rounds = 50, 
#     eval_set = evals
)

# LGBM
lgb = LGBMRegressor(
    n_estimators = 700,
    max_depth = 30,
    num_leaves = 31,
    learning_rate = 0.02,
    colsample_bytree = 0.3,
    subsample = 0.8,
    min_child_samples = 75,
    reg_alpha = 3.2,
    reg_lambda = 8.4)

In [46]:
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y-log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

In [48]:
# StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score_lst = []

for train_idx, test_idx in tqdm(cv.split(X_train_res, y_train), total=10):
    x_tr = X_train_res.iloc[train_idx]
    y_tr = y_train.iloc[train_idx]
    
    x_val = X_train_res.iloc[test_idx]
    y_val = y_train.iloc[test_idx]
    
    
    model = xgb
    model.fit(x_tr, y_tr, eval_metric='rmsle', verbose=False)

    pred = model.predict(x_val)
    score = rmsle(y_val, pred)
    score_lst.append(score)

print('each fold rmsle : ', score_lst)
print('rmsle : ', np.mean(score_lst))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]

each fold rmsle :  [0.46160381514472215, 0.46017688554274394, 0.46234198474409965, 0.4571881411232892, 0.45982457381381087, 0.459372426417145, 0.45831563354564175, 0.4609091291184987, 0.4584216245706154, 0.4607363135852191]
rmsle :  0.45988905276057856


In [49]:
# StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score_lst = []

for train_idx, test_idx in tqdm(cv.split(X_train_res, y_train), total=10):
    x_tr = X_train_res.iloc[train_idx]
    y_tr = y_train.iloc[train_idx]
    
    x_val = X_train_res.iloc[test_idx]
    y_val = y_train.iloc[test_idx]
    
    
    model = lgb
    model.fit(x_tr, y_tr, eval_metric='rmsle')

    pred = model.predict(x_val)
    score = rmsle(y_val, pred)
    score_lst.append(score)

print('each fold rmsle : ', score_lst)
print('rmsle : ', np.mean(score_lst))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.18s/it]

each fold rmsle :  [0.460243725604073, 0.45854501276523185, 0.45805185707455454, 0.4569000668338597, 0.45498905800539186, 0.45860417710491097, 0.4578057519932152, 0.46082521059744674, 0.4596101353843705, 0.4609782506674754]
rmsle :  0.45865532460305297
